# Prerequisite: locating AES phases and Subbyte operations

In this prerequisite part, we will first try to identify the different phases of the AES corresponding to the main functions in one round: AddRoundKey, SubBytes, ShiftRow and MixColumns. We will then identify the timing of the different SubBytes operations for the first round. 

First, we will begin by looking at the aspect of a round for the original code. Let us start by compiling the AES and programming the Target Board.

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEXMEGA'
CRYPTO_TARGET = 'TINYAES128C'

In [ ]:
%run "Helper_Scripts/Setup_Generic.ipynb"

In [ ]:
fw_path = "../hardware/victims/firmware/simpleserial-aes/simpleserial-aes-{}.hex".format(PLATFORM)

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../hardware/victims/firmware/simpleserial-aes
make PLATFORM=$1 CRYPTO_TARGET=$2

For now, only one trace is required:

In [ ]:
cw.program_target(scope, prog, fw_path)

We can plot it using Bokeh:

In [ ]:
# Capture Trace
from tqdm import tnrange
import numpy as np
import time

ktp = cw.ktp.Basic()

key, text = ktp.next() # automated creation of a key and text pair 

scope.adc.offset = 1500
scope.adc.samples = 1000
trace = cw.capture_trace(scope, target, text, key)

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()
p = figure(plot_width = 1000, plot_height = 600)

xrange = range(len(trace.wave))
p.line(xrange, trace.wave, line_color = "purple")
show(p)

Modify now the file hardware/victims/firmware/crypto/tiny-AES128-C/aes.c containing the AES function. Add nops between thedifferent phases of a round (as already done in TME1). 

Recompile, recapture one trace and identify the different functions. In particular, you should identify the SBox computation with certainty. Note somewhere how you should adjust the `offset` and `samples` parameters of the capture so that the captured trace contains the full first round SBox computation (because all bytes will be attacked during the TME), but no other part of the computation (so as to minimize the risk of having other samples randomly giving better results for a wrong key than the correct key). 

Once it is done, try to identify (using nop operations) the offset and number of samples required for capturing each of the 16 operations performed inside the first SBox computation. Note precisely these offset and number of samples, you'll have to set as parameters in order to fully capture one of these operations but not the other ones.

The `offset` and `samples` values you found should correspond to a code in which there is no `nop` instruction.

# Introduction to DPA & HW Assumption

Remove the `nops` of the AES code if not already done and recompile the AES.

## DPA Attack Theory

Going back to the theory, remember that we have an assumed relationship between power on the data lines and measured power consumption. You can see this in the following:

![Power lines](img/dpa_4bits_powerhw_scaled.png)

How do we prove this is true? Let's plot the Hamming weight (HW) of the data to figure this out along with the power traces! We are going to use the AES algorithm (it doesn't matter what we use), because you've just analyzed it and because this is the target of the DPA attack we'll perform at the end of this TME.

## Capturing Power Traces

Capturing power traces will be very similar to previous tutorials, except this time we'll be using a loop to capture multiple traces, as well as numpy to store them. You'll easily find using Internet some documentation about numpy (numpy arrays, operations on numpy arrays, etc.) if needed. 

It's not necessary, but we'll also plot the trace we get using `bokeh`.

### Capturing Traces

Below you can see the capture loop. The main body of the loop loads some new plaintext, arms the scope, sends the key and plaintext, then finally records and appends our new trace to the `traces[]` list. At the end, we convert the trace data to numpy arrays, since that's what we'll be using for analysis.

In [ ]:
# Capture Traces
from tqdm import tnrange
import numpy as np
import time

ktp = cw.ktp.Basic() # object dedicated to the generation of key (fixed by default) and plain text

traces = [] # list of traces
N = 1000  # Number of traces

for i in tnrange(N, desc = 'Capturing traces'):
    key, text = ktp.next()  # creation of a pair comprising (fixed) key and text 

    trace = cw.capture_trace(scope, target, text, key) # a trace is composed of the following fields :
                                                       #    a wave (samples)
                                                       #    textin (input text), textout (output text)
                                                       #    key (input key)
    
    if trace is None:
        continue
    traces.append(trace)

# Convert traces to numpy arrays
trace_array = np.asarray([trace.wave for trace in traces])  # if you prefer to work with numpy array for number crunching
textin_array = np.asarray([trace.textin for trace in traces])
known_keys = np.asarray([trace.key for trace in traces])    # for fixed key generation, these keys are all the same

Now that we have our traces, we can also plot them using Bokeh:

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()
p = figure(plot_width = 1000, plot_height = 300)

xrange = range(len(traces[0].wave))
p.line(xrange, traces[0].wave, line_color = "red")
show(p)

In [ ]:
# cleanup the connection to the target and scope
#scope.dis()
#target.dis()

## Trace Analysis

### Using the Trace Data

Now that we have some traces, let's look at what we've actually recorded. Looking at the earlier parts of the script, we can see that the trace data is in `trace_array`, while `textin_array` stores what we sent to our target to be encrypted. For now, let's get some basic information (the total number of traces, as well as the number of sample points in each trace) about the traces, since we'll need that later:

In [ ]:
numtraces = np.shape(trace_array)[0] #total number of traces
numpoints = np.shape(trace_array)[1] #samples per trace

print(numtraces)
print(numpoints)

For the analysis, we'll need to loop over every byte in the key we want to attack, as well as every trace:
```python
for bnum in range(0, 16):
    for tnum in range(0, numtraces):
        pass
```
Though we didn't loop over them, note that each trace is made up of a bunch of sample points.
Let's take a closer look at AES so that we can replace that `pass` with some actual code.

### Calculating Hamming Weight (HW) of Data

Now that we have some power traces of our target that we can use, we can move on to the next steps of our attack. Looking way back to how AES works, remember we are effectively attemping to target the position at the bottom of this figure:

![S-Box HW Leakage Point](img/Sbox_cpa_detail.png)

The objective is thus to determine the output of the S-Box, where the S-Box is defined as follows:

In [ ]:
numtraces = np.shape(trace_array)[0] #total number of traces
numpoints = np.shape(trace_array)[1] #samples per trace

sbox = (
    0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76,
    0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0,
    0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15,
    0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75,
    0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84,
    0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf,
    0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8,
    0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2,
    0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73,
    0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb,
    0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79,
    0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08,
    0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a,
    0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e,
    0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf,
    0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16)

def intermediate(pt, keyguess):
    return sbox[pt ^ keyguess]

HW = [bin(n).count("1") for n in range(0, 256)]

def intermediate(pt, key):
    return sbox[pt ^ key]

#Example - PlainText is 0x12, key is 0xAB
HW[intermediate(0x12, 0xAB)]

Thus we need to write a function taking a single byte of input, a single byte of the key, and return the output of the S-Box:

In [ ]:
def intermediate(pt, keyguess):
    return sbox[pt ^ keyguess]


Finally, remember we want the Hamming Weight of the S-Box output. Our assumption is that the system is leaking the Hamming Weight of the output of that S-Box. As a dumb solution, we could first convert every number to binary and count the 1's:

```python
>>> bin(0x1F)
'0b11111'
>>> bin(0x1F).count('1')
5
```
This will ultimately be fairly slow. Instead we make a lookup table named `HW` using this idea:

In [ ]:
HW = [bin(n).count("1") for n in range(0, 256)]

def intermediate(pt, key):
    return sbox[pt ^ key]

#Example - PlainText is 0x12, key is 0xAB
HW[intermediate(0x12, 0xAB)]

### Plotting HW

Finally, what we are going to do is plot each of the different "classes" in a different color. With this we should see if there is some location that has relatively obvious difference in Hamming weight. We get that easily using the `HW` array and `intermediate()` function we defined earlier and a loop to plot all of the traces.

To make this easier, we can zoom in on some specific area. In the following example a small subset of the full capture is to be plotted only. You can more easily figure out what this point should be by using the CPA attack (we'll talk about later) which provides more information about where the leakage is happening. For now let's pretend we know already what a "good" point is by considering the part of the trace that corresponds to the SBox computation for a given byte of the key. You'll have to adapt the `plot_start` and `plot_end` using your findings in the prerequisite steps. You must test with different key bytes (e.g. key byte 0, key byte 1, key byte 15)

At the correct sample, you should observe a smooth gradation of red (from light to dark), as those colours are associated w.r.t. to the HW of the targeted value. 

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.palettes import brewer

output_notebook()
p = figure(plot_width = 1000, plot_height = 400)

#Must run S-Box() script first to define the HW[] array and intermediate() function

# This is the most compelling byte we found for this part
bnum = 2 # change bnum to play with different key byte (from 0 to 15)

plot_start =  0 # set here the sample where to start for the selected key byte
plot_end =  200  # set here the sample where to end for the selected key byte

xrange = range(len(traces[0].wave))[plot_start:plot_end]

color_mapper = (brewer['Reds'][9])
print(color_mapper)

for trace in traces:
    hw_of_byte = HW[intermediate(trace.textin[bnum], trace.key[bnum])]
    p.line(xrange, trace.wave[plot_start:plot_end], line_color=color_mapper[hw_of_byte])

show(p)

### Finding Average at Locations

So with an idea that there are differences, let's actually plot them to see how "linear" they are in real life. We're going to pick a point (again), and use that to get the averages. The following will find and print the averages:

In [ ]:
import numpy as np

# This is the associated avg_point we found for the first byte
avg_point =  36 # set here the sample at which you made the correct observation in the previous section

hw_list = [ [], [], [], [], [], [], [], [], []]
for trace in traces:
    hw_of_byte = HW[intermediate(trace.textin[bnum], trace.key[bnum])]
    hw_list[hw_of_byte].append(trace.wave[avg_point])
    
hw_mean_list = [np.mean(hw_list[i]) for i in range(0, 9)]

for hw in range(1, 9):
    print(hw_mean_list[hw])
    print("HW " + str(hw) + ": " + str(hw_mean_list[hw]))

If you have the correct point, the above should look somewhat linear. Let's get a nice plot of this to see it visually:

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()
p = figure(title="HW vs Voltage Measurement")
p.line(range(1, 9), hw_mean_list[1:9], line_color="red")
p.xaxis.axis_label = "Hamming Weight of Intermediate Value"
p.yaxis.axis_label = "Average Value of Measurement"
show(p)

That's it! You should see a nice linear plot as a result. If not you might have selected the wrong point, change it if needed. Whatever, you might notice the slope is opposite what you expect.

This happens for a good reason. If you remember how we are measuring the current into the device, you'll find out that the voltage will go DOWN for an INCREASE in current. You can see this in the following figure:

![Measurepoint point](img/vmeasure.png)

We are measuring the drop across the shunt resistor. An increase in the current causes a higher voltage across the resistor. When no current flows there is no drop across the resistor. But since we only measure a single end of the resistor, we see a higher voltage when no current flows.

We can fix the slope by simply inverting the measurement direction (adding a - in front of the measurement).

# Differential Power Analysis Attack

## DPA Attack Theory

As we have seen earlier, the Hamming Weight of the result of the SBox operation in AES has a measurable effect on the power consumed by the microcontroller. It turns out that just this effect (and not anything stronger, such as its linearity) is enough information to break an AES key. There's a few different ways we could go about this, but for this tutorial, we'll be looking at difference of means. With this technique, the goal is to separate the traces by one or several bit(s) in the result of the SBox output.

Considering one bit, the group of traces for which its value is 1 should, on average, have higher power consumption during the SBox operation than the other set in which its value is 0.

For multi-bit DPA, one must find a way to combine the values of the different bits to make two relevant groups.

Whether or not we get a large difference in the means between these two groups depends on whether they were properly sorted into these groups. If not, there should be, on average, little difference between the two and therefore a low difference of means. Recall the SBox operation:

![title](https://wiki.newae.com/images/7/71/Sbox_cpa_detail.png)

The SBox output depends on the subkey, which we don't know (and the plaintext, which we do). However, since there's a large difference of means for the correct key and small ones for the rest of the possible subkeys, we have a method of checking whether a given subkey is correct. If we calculate the difference of means for each subkey, the correct one will have the largest difference of means.

Thus, the attack consists of the following steps:
1. Capture power traces with varying plaintext (and fixed key !)
1. Group each trace by the value of their SBox output's for a given subkey guess
1. Calculate the difference of means (this is a trace)
1. Repeat for each possible subkey
1. Select the largest difference of means -> this should be the correct subkey
1. Repeat for each subkey in the key

At the end, we should get a correct AES key!

## Performing the DPA

Capture a big set of traces (typically 5000) using the same way as in the previous part of this practical, then write a python code that performs the attack on the first key byte before extending this code to attack all the key bytes. You'll write some comments to explain your code (and your implemented attack)

In [ ]:
# Capture Traces
from tqdm import tnrange
import numpy as np
import time
import os

traces_dir = 'traces'

ktp = cw.ktp.Basic() # object dedicated to the generation of key (fixed by default) and plain text

traces = [] # list of traces
N = 5000  # Number of traces

scope.adc.offset = 1500
scope.adc.samples = 1000

print("key = ", key)

for i in tnrange(N, desc = 'Capturing traces'):
    key, text = ktp.next()  # creation of a pair comprising (fixed) key and text

    trace = cw.capture_trace(scope, target, text, key) # a trace is composed of the following fields :
                                                       #    a wave (samples)
                                                       #    textin (input text), textout (output text)
                                                       #    key (input key)
    
    if trace is None:
        continue
    traces.append(trace)

f = open(os.path.join(traces_dir, 'key.raw'), "wb")
f2 = open(os.path.join(traces_dir, 'plain.raw'), "wb")
f3 = open(os.path.join(traces_dir, 'traces.raw'), "wb")

# same key for all plaintext so write only once
print("key   : ",traces[0].key)
for b in traces[0].key:
        f.write(np.uint8(b))

# Write all the plaintexts and waves to "plain.raw" and "traces.raw"
for trace in traces:
    print("plain : ",trace.textin)
    for b in trace.textin:
        f2.write(np.uint8(b))
    for b in trace.wave:
        f3.write(b)
        
f.close()
f2.close()
f3.close()

# Convert traces to numpy arrays
trace_array = np.asarray([trace.wave for trace in traces])  # if you prefer to work with numpy array for number crunching
textin_array = np.asarray([trace.textin for trace in traces])
known_keys = np.asarray([trace.key for trace in traces])    # for fixed key generation, these keys are all the same

numtraces = np.shape(trace_array)[0] #total number of traces
numpoints = np.shape(trace_array)[1] #samples per trace
print("numtraces = ", numtraces)
print("numpoints = ", numpoints)

sbox = (
    0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76,
    0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0,
    0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15,
    0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75,
    0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84,
    0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf,
    0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8,
    0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2,
    0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73,
    0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb,
    0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79,
    0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08,
    0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a,
    0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e,
    0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf,
    0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16)

def intermediate(pt, keyguess):
    return sbox[pt ^ keyguess + 1]

HW = [bin(n).count("1") for n in range(0, 256)]

def intermediate(pt, key):
    return sbox[pt ^ key]

#Example - PlainText is 0x12, key is 0xAB
HW[intermediate(0x12, 0xAB)]

final_key = np.zeros((1, 16))

# Time measure for the attack
t0 = time.clock()

for bnum in range(0, 16):
    # Init a buffer of length 256, max value of a byte, and initialize it
    group = np.zeros(256)
    for k in range(256):
        # Two separate groups to discriminate the traces
        grp1 = np.zeros(numpoints)
        nb_traces_g1 = 0
        
        grp2 = np.zeros(numpoints)
        nb_traces_g2 = 0
        
        """
        The next part calculate the means iterativaly. However,
        it requires a first loop to calculate the length of the
        two sub-groups, as we don't know know it firsthand.
        The second loop then calculates the means iteratively of
        the two sub-groups.

        Using this method reduces the cancellation phenomena, but
        increases the time taken overall by a slight amount.
        """
        for trace in traces:
            hw_of_byte = HW[intermediate(trace.textin[bnum], k)]
            # Discriminate over the second to last bit
            first_byte = hw_of_byte & 0b0000001
            if first_byte == 1:
                nb_traces_g1 += 1
            else:
                nb_traces_g2 += 1
            
        for trace in traces:
            hw_of_byte = intermediate(trace.textin[bnum], k)
            first_byte = hw_of_byte & 0b0000001
            
            # Calculate each mean iteratively
            if first_byte == 1:
                grp1 = np.add(grp1, np.true_divide(np.subtract(trace.wave, grp1), nb_traces_g1))
            else:
                grp1 = np.add(grp2, np.true_divide(np.subtract(trace.wave, grp2), nb_traces_g2))
        
        tmp = np.abs(np.subtract(grp1, grp2))
        group[k] = np.amax(tmp)
    
    # Get the indices of maximum element in numpy array
    result = np.where(group == np.amax(group))
    
    final_key[0][bnum] = result[0];

print("Time taken = ", time.clock(), " seconds.")
print(final_key)
for i in final_key[0]:
    print(hex(int(i))," " , end = '')

print()
for i in range(16):
    print(int(final_key[0][i]) - key[i],", ", end='')

## Tips

* You can use the function `numpy.save(filename, array)` in order to save your traces in a file; this avoids the need to capture traces every time you want to work on the attack code
* In order to recover data saved, you have to use `array = numpy.load(filename)`
* If the attack does not work, you can try to modify gain settings, or to increase the number of traces you capture - this attack is much more sensitive to gain settings and noise than a CPA attack would be. Also, an attack involving all the bits of the bytes gives better results than an attack using only one bit.
* Depending on your board you may not be able to break all the key bytes, let us know if you experience such a case (we'll try to provide you another board if we can, at least let you test with another one, if the board is the real root of the problem !)

## Conclusion

You have (hopefully) broken AES using a DPA attack! As you might have discovered during this tutorial, there can be quite a few issues with the difference of means method for breaking AES keys:

* It's quite susceptible to noise
* If you don't select the right `offset` and `samples` the attack can easily pick up other parts of the AES operation (you can try with different values to see this point)
* The attack typically requires a lot of traces. These software unprotected AES implementations are pretty weak against power analysis, but they still required thousands of traces to break

Nevertheless, using a difference of means attack can still be very useful in several contexts.

**Important** You must write a small report in which you must explain the different steps of this TME, in particular how you implemented the DPA attack. You must also give and discuss some results: it is recommended to give some statistics about the success of the attack according to the selected byte (and bit) and the number of traces. Note that you can change (manually or not) the key to test with different ones ! 

In [ ]:
scope.dis()
target.dis()